In [ ]:
import chess.pgn
import io
import json

In [45]:
# 1. Add fens to files
file_no_fens = './openings-no-variations.json'
output_file = 'openings-no-variations-fen.json'

f = open(file_no_fens)
openings = json.load(f)
for opening in openings:
    pgn = opening['pgn']
    game = chess.pgn.read_game(io.StringIO(pgn))
    board = game.board()
    for move in game.mainline_moves():
        board.push(move)
    opening['fen'] = board.fen()

# dump to file
with open(output_file, 'w') as f:
    json.dump(openings, f, indent=4, ensure_ascii=True)

In [ ]:
# TODO: 2. using file contains FENs, call lichess api, appending winrates
# see ./openings_novar.json 

In [ ]:
# 3. 
# - Filter out openings with identical names, taking the smallest PGN string (more generic opening PGN)
# - Sort the list by popularity: total number of games from smallest to largest